In [1]:
!module load cuda

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as T

import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from jupyterplot import ProgressPlot
from tqdm import tqdm, trange

In [3]:
class WeatherDataset(Dataset):
    def __init__(self, weather_data = 'data/arpafvg_fllbandiera_clean.csv', img_dir = 'data/images_enhanced'):
        initial_data=pd.read_csv(weather_data)
        #normalize the data
        for col in initial_data.columns:
            if col != 'giorno' and col != 'mese' and col != 'anno':
                initial_data[col] = (initial_data[col] - initial_data[col].mean()) / initial_data[col].std()
        self.weather_data = initial_data
        self.img_dir = img_dir

    def __getitem__(self, date):
        #get day month year from date in format dd-mm-yyyy
        day, month, year = date.split('_')
        #get from weather data the row with the same date
        weather_data = self.weather_data[(self.weather_data['giorno'] == int(day)) & (self.weather_data['mese'] == int(month)) & (self.weather_data['anno'] == int(year))]
        weather_data = torch.tensor(weather_data.values[0])

        image = os.path.join(self.img_dir, str(date) + '.jpg')  
        image = read_image(image)   
        return [image, weather_data]
    
    def __len__(self):
        return len(self.weather_data)

In [4]:
dataset = WeatherDataset()

In [5]:
#generate a list of strings dd-mm-yyyy from 01-06-2023 to 15-6-2024
from datetime import datetime, timedelta
start_date = datetime.strptime("01_01_2022", "%d_%m_%Y")
end_date = datetime.strptime("15_06_2024", "%d_%m_%Y")
date_generated = [start_date + timedelta(days=x) for x in range(0, (end_date-start_date).days)]
#transform the list of datetime objects in a list of strings in format dd-mm-yyyy
date_generated = [date.strftime("%d_%m_%Y") for date in date_generated]


In [6]:
train_len = int(0.8 * len(date_generated))
test_len = len(date_generated) - train_len
#split the dataset: date_trainset contains the first 80% of the dates, date_testset contains the remaining 20%
date_trainset = date_generated[:train_len]
date_testset = date_generated[train_len:]

In [7]:
(image, weather_data) = dataset[date_generated[np.random.randint(low = 0, high = len(dataset))]]
#print(weather_data)
#print(image)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [8]:
class DeepWeather(nn.Module):
    def __init__(self):
        super(DeepWeather, self).__init__()

        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = (3, 3), stride = 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p = 0.3)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = (3, 3), stride = 2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = (3, 3), stride = 2)
        self.bn3 = nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(6287, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, inputs):
        x1, x2 = inputs[0], inputs[1]
        

        x1 = self.bn1(self.dropout(self.pool(F.leaky_relu(self.conv1(x1)))))
        x1 = self.bn2(self.dropout(self.pool(F.leaky_relu(self.conv2(x1)))))
        x1 = self.bn3(self.dropout(self.pool(F.leaky_relu(self.conv3(x1)))))
        x1 = torch.flatten(x1, start_dim = 1)

        x = torch.cat((x1, x2), dim = 1)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [9]:
def train(model, dataset, date_trainloader, date_testloader, criterion, optimizer, epochs, first_time = True, num_saved_epochs = 0):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    train_loss_epochs = []
    test_loss_epochs = []
    bar = trange(epochs, desc=f"Epoch ?/?, Train Loss: ?, Test Loss: ?")
    for epoch in bar:
        model.train()
        train_losses = []
        for date_inputs in date_trainloader:
            inputs = dataset[date_inputs[0]]
            inputs = [inputs[0].float().unsqueeze(0).to(device), inputs[1].float().unsqueeze(0).to(device)]

            #increment of 1 day the date_inputs
            date_next_day = date_inputs[0].split('_')
            date_next_day = datetime(int(date_next_day[2]), int(date_next_day[1]), int(date_next_day[0])) + timedelta(days = 1)
            date_next_day = date_next_day.strftime("%d_%m_%Y")
            target=dataset[date_next_day][1][3].float().to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), target)
            #convert the loss type to float
            loss = loss.float()
            #print(f'loss is {loss}\n')
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        train_loss_epochs.append(np.mean(train_losses))
        model.eval()
        test_losses = []
        for date_inputs in date_testloader:
            inputs = dataset[date_inputs[0]]
            inputs = [inputs[0].float().unsqueeze(0).to(device), inputs[1].float().unsqueeze(0).to(device)]
            outputs = model(inputs)

            #increment of 1 day the date_inputs
            date_next_day = date_inputs[0].split('_')
            date_next_day = datetime(int(date_next_day[2]), int(date_next_day[1]), int(date_next_day[0])) + timedelta(days = 1)
            date_next_day = date_next_day.strftime("%d_%m_%Y")
            target=dataset[date_next_day][1][3].float().to(device)

            loss = criterion(outputs.squeeze(), target)
            test_losses.append(loss.item())
        test_loss_epochs.append(np.mean(test_losses))
        bar.set_description(f"Epoch {epoch + 1}/{epochs}, Train Loss: {np.mean(train_losses)}, Test Loss: {np.mean(test_losses)}")
        #if first_time:
            #torch.save(model.state_dict(), f"deepweather_epoch{num_saved_epochs + epoch + 1}.pth")

    return train_loss_epochs, test_loss_epochs

In [10]:
#Uncomment the lines below if you want to train/load a pretrained model
#num_saved_epochs = 50
#model = model.load_state_dict(torch.load(f'weights/epoch_{num_saved_epochs}'))

#Comment the (ONE) line below if you want to train/load a pretrained model.
#!mkdir weights
model = DeepWeather()

date_trainloader = DataLoader(date_trainset, batch_size = 1, shuffle = False)
date_testloader = DataLoader(date_testset, batch_size = 1, shuffle = False)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 10e-6, momentum = 0)

In [11]:
train_losses, val_losses = train(model, dataset, date_trainloader, date_testloader, criterion, optimizer, epochs = 50, first_time = True, num_saved_epochs = 0)

Epoch ?/?, Train Loss: ?, Test Loss: ?:   0%|                                                    | 0/50 [00:00<?, ?it/s]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
epochs = [epoch for epoch in range(50)]
plt.plot(epochs, train_losses, label = 'Training Loss')
plt.plot(epochs, val_losses, label = 'Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
def predict(model, date):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    inputs = dataset[date]
    inputs = [inputs[0].float().unsqueeze(0).to(device), inputs[1].float().unsqueeze(0).to(device)]

    outputs = model(inputs)    
    
    return outputs

In [ ]:
print(predict(model, '31_12_2023'))
print(dataset['31_12_2023'][1][3])

In [ ]:
for date in date_testset:
    predicted_rain = predict(model, date)
    #non normalized results
    predicted_rain = predicted_rain 
    print(f"Predicted rain normalized for {date} is {predicted_rain}")
    #while real rain is
    print(f"Real rain normalized  for {date} is {dataset[date][1][3]}")   


In [ ]:
data = valset[np.random.randint(low = 0, high = len(valset))]

inputs = data[0]
inputs[0] = inputs[0].unsqueeze(0)
inputs[1] = inputs[1].unsqueeze(0)

label = (data[1] * dataset.labels_std) + dataset.labels_means
forecast = (inputs[1] * dataset.forecasts_std) + dataset.forecasts_means

prediction = predict(model, dataset, inputs)

forecast = forecast.squeeze()
prediction = prediction.squeeze()

print(f"Forecast:\n    Average Temp: {forecast[0]:.2f}K    Min Temp: {forecast[1]:.2f}K    Max Temp: {forecast[2]:.2f}K    Humidity: {forecast[4]:.2f}%    Clouds: {forecast[3]:.2f}%")
print(f"\nLabel:\n    Average Temp: {label[0]:.2f}K    Min Temp: {label[1]:.2f}K    Max Temp: {label[2]:.2f}K    Humidity: {label[3]:.2f}%")
print(f"\nPrediction:\n    Average Temp: {prediction[0]:.2f}K    Min Temp: {prediction[1]:.2f}K    Max Temp: {prediction[2]:.2f}K    Humidity: {prediction[3]:.2f}%")

In [ ]:
trainloader = DataLoader(trainset, batch_size = 128, shuffle = False)
train_r2 = 0
train_avg_temp_r2 = 0
train_min_temp_r2 = 0
train_max_temp_r2 = 0
train_humidity_r2 = 0
for i, data in enumerate(trainloader):
    inputs = data[0]
    labels = data[1]
    labels = (labels * dataset.labels_std) + dataset.labels_means
    
    outputs = predict(model, dataset, inputs)
    
    train_r2 += r2_score(labels, outputs)
    train_avg_temp_r2 += r2_score(labels[:, 0], outputs[:, 0])
    train_min_temp_r2 += r2_score(labels[:, 1], outputs[:, 1])
    train_max_temp_r2 += r2_score(labels[:, 2], outputs[:, 2])
    train_humidity_r2 += r2_score(labels[:, 3], outputs[:, 3])
    
train_r2 /= len(trainloader)
train_avg_temp_r2 /= len(trainloader)
train_min_temp_r2 /= len(trainloader)
train_max_temp_r2 /= len(trainloader)
train_humidity_r2 /= len(trainloader)

print("Training Data R2 Scores:")
print(f"    Avg Temp: {train_avg_temp_r2:.3f}")
print(f"    Min Temp: {train_min_temp_r2:.3f}")
print(f"    Max Temp: {train_max_temp_r2:.3f}")
print(f"    Humidity: {train_humidity_r2:.3f}")
print(f"    Total: {train_r2:.3f}")
    
    
    
    
valloader = DataLoader(valset, batch_size = 128, shuffle = False)
val_r2 = 0
val_avg_temp_r2 = 0
val_min_temp_r2 = 0
val_max_temp_r2 = 0
val_humidity_r2 = 0
for i, data in enumerate(valloader):
    inputs = data[0]
    labels = data[1]
    labels = (labels * dataset.labels_std) + dataset.labels_means
    
    outputs = predict(model, dataset, inputs)
    
    val_r2 += r2_score(labels, outputs)
    val_avg_temp_r2 += r2_score(labels[:, 0], outputs[:, 0])
    val_min_temp_r2 += r2_score(labels[:, 1], outputs[:, 1])
    val_max_temp_r2 += r2_score(labels[:, 2], outputs[:, 2])
    val_humidity_r2 += r2_score(labels[:, 3], outputs[:, 3])
    
val_r2 /= len(valloader)
val_avg_temp_r2 /= len(valloader)
val_min_temp_r2 /= len(valloader)
val_max_temp_r2 /= len(valloader)
val_humidity_r2 /= len(valloader)

print("\nValidation Data R2 Scores:")
print(f"    Avg Temp: {val_avg_temp_r2:.3f}")
print(f"    Min Temp: {val_min_temp_r2:.3f}")
print(f"    Max Temp: {val_max_temp_r2:.3f}")
print(f"    Humidity: {val_humidity_r2:.3f}")
print(f"    Total: {val_r2:.3f}")




dataloader = DataLoader(dataset, batch_size = 128)
forecast_r2 = 0
forecast_avg_temp_r2 = 0
forecast_min_temp_r2 = 0
forecast_max_temp_r2 = 0
forecast_humidity_r2 = 0
for i, data in enumerate(dataloader):
    inputs = data[0]
    labels = data[1]
    forecasts = inputs[1]
    forecasts = forecasts[:, np.r_[:3, 4]]
    
    forecast_avg_temp_r2 += r2_score(labels[:, 0], forecasts[:, 0])
    forecast_min_temp_r2 += r2_score(labels[:, 1], forecasts[:, 1])
    forecast_max_temp_r2 += r2_score(labels[:, 2], forecasts[:, 2])
    forecast_humidity_r2 += r2_score(labels[:, 3], forecasts[:, 3])
    forecast_r2 += r2_score(forecasts, labels)
    
forecast_r2 /= len(dataloader)
forecast_avg_temp_r2 /= len(dataloader)
forecast_min_temp_r2 /= len(dataloader)
forecast_max_temp_r2 /= len(dataloader)
forecast_humidity_r2 /= len(dataloader)

print("\nForecast Data R2 Scores:")
print(f"    Avg Temp: {forecast_avg_temp_r2:.3f}")
print(f"    Min Temp: {forecast_min_temp_r2:.3f}")
print(f"    Max Temp: {forecast_max_temp_r2:.3f}")
print(f"    Humidity: {forecast_humidity_r2:.3f}")
print(f"    Total: {forecast_r2:.3f}")